# Hyperparameter Tuning using HyperDrive

In this notebook we are going to train a model using and adjust its hyperparameters using Hyperdrive.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

## Dataset

The dataset we will be using is the "Heart failure prediction dataset" from Kaggle (https://www.kaggle.com/fedesoriano/heart-failure-prediction). This dataset tries to help in the early detection of severe heart diseases by studying the way several health indicators affect the occurrence of such diseases. This dataset is a combination of 5 different datasets about this kind of diseases (more information in the Kaggle url provided earlier). 

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacityproj3hyper'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: quick-starts-ws-165366
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-165366


In [3]:
cluster_name="udacityprojclust"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('cpu cluster already exist. Using it.')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

cpu cluster already exist. Using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

The model we will be using it is a simple Logistic Regression, this has the advantage of being quite interpretable. We will be using also hyperdrive to configure the best hyperparameters from this kind of model.
For the optimization of hyperparameters we will use random parameter sampling that provides a quick and robust way of searching the appropiate parameters, we will also implement an early stopping policy for optimizing the use of computational resources, the early stopping policy is the BanditPolicy that gets a right trade-off between saving compute resources and losing some probable good cases that would have improved the primary metric.

In [5]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'C': uniform(0.5,1.5), 'max_iter': choice(50, 75, 100, 125, 150)})

# Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', entry_script='train.py', compute_target=cpu_cluster, script_params={'--C': 'C', '--max_iter':'max_iter'})

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, policy=early_termination_policy, primary_metric_name='AUC', max_duration_minutes=100, max_total_runs=10, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, estimator=estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [20]:
! python ./train.py --C 0.5 --max_iter 150

Attempted to log scalar metric Regularization Strength::
0.5
Attempted to log scalar metric Max iterations::
150
Attempted to log scalar metric AUC:
0.8179563492063492


In [18]:
# Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In [19]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In this next cells we are going to get the best model measured by the primary metric chosen previously. We will get also the best hyperparameters from the hyperdrive run and finally we will save it to the Azure storage.

In [ ]:
# Get the best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_param = best_run.get_details()['runDefinition']['arguments']

print('Best parameters')
print(best_param)

LR_model = LogisticRegression(C=float(best_param[1]), max_iter=int(best_param[3]))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

LR_model.fit(x_train, y_train)

In [ ]:
# Save the best model

joblib.dump(LR_model, 'hyperdrive_LR.model')

## Model Deployment

We register the model in our Azure workspace in order to get the chance to deploy it if we need it.

In [ ]:
# Register de model

model = Model.register(workspace=ws,
                       model_name='my-udacityproj3-hyperdrivemodel', # Name of the registered model in your workspace.
                       model_path='./hyperdrive_LR.model',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.ScikitLearn,  # Framework used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1.0),
                       description='Hyperdrive best model for heart disease prediction.',
                       tags={'area': 'heartdisease', 'type': 'classification'})

print('Name:', model.name)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

